In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from importlib import reload
import logging
reload(logging)
logging.basicConfig(format='%(asctime)s %(levelname)s:%(message)s', level=logging.INFO, datefmt='%H:%M:%S')

In [3]:
import sys

sys.path.insert(0, '../..')

## Load Dataset

In [4]:
from entity_embed.benchmarks import AmazonGoogleBenchmark

benchmark = AmazonGoogleBenchmark(data_dir_path="../data/")
benchmark

11:39:43 INFO:Extracting Amazon-Google...
11:39:43 INFO:Reading Amazon-Google row_dict...
11:39:43 INFO:Reading Amazon-Google train.csv...
11:39:43 INFO:Reading Amazon-Google valid.csv...
11:39:43 INFO:Reading Amazon-Google test.csv...


<AmazonGoogleBenchmark> from http://pages.cs.wisc.edu/~anhai/data1/deepmatcher_data/Structured/Amazon-Google/amazon_google_exp_data.zip

## Preprocess

In [5]:
attr_list = ['title', 'manufacturer', 'price']

In [6]:
from tqdm.auto import tqdm
import unidecode
import itertools
from entity_embed import default_tokenizer

def clean_str(s):
    s = unidecode.unidecode(s).lower().strip()
    s_tokens = itertools.islice((s_part[:30] for s_part in default_tokenizer(s)), 0, 30)
    return ' '.join(s_tokens)[:300]

for row in tqdm(benchmark.row_dict.values()):
    for attr in attr_list:
        row[attr] = clean_str(row[attr])

## Init Data Module

In [7]:
import torch
import numpy as np

random_seed = 42
torch.manual_seed(random_seed)
np.random.seed(random_seed)

In [8]:
alphabet = list('0123456789abcdefghijklmnopqrstuvwxyz!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~ ')

In [9]:
attr_info_dict = {
    'title': {
        'field_type': "SEMANTIC_MULTITOKEN",
        'tokenizer': "entity_embed.default_tokenizer",
        'vocab': "fasttext.en.300d",
        'use_mask': True,
    },
    'manufacturer': {
        'field_type': "MULTITOKEN",
        'tokenizer': "entity_embed.default_tokenizer",
        'alphabet': alphabet,
        'use_mask': True,
        'max_str_len': None,  # compute
    },
    'price': {
        'field_type': "STRING",
        'alphabet': alphabet,
        'max_str_len': None,  # compute
    }
}

In [10]:
from entity_embed import AttrInfoDictParser

row_numericalizer = AttrInfoDictParser.from_dict(attr_info_dict, row_dict=benchmark.row_dict)
row_numericalizer.attr_info_dict

11:39:44 INFO:Loading vectors from .vector_cache/wiki.en.vec.pt
11:39:47 INFO:For attr=manufacturer, computing actual max_str_len
11:39:47 INFO:actual_max_str_len=15 must be pair to enable NN pooling. Updating to 16
11:39:47 INFO:For attr=manufacturer, using actual_max_str_len=16
11:39:47 INFO:For attr=price, computing actual max_str_len
11:39:47 INFO:actual_max_str_len=11 must be pair to enable NN pooling. Updating to 12
11:39:47 INFO:For attr=price, using actual_max_str_len=12


{'title': NumericalizeInfo(field_type=<FieldType.SEMANTIC_MULTITOKEN: 'semantic_multitoken'>, tokenizer='entity_embed.data_utils.numericalizer.default_tokenizer', alphabet=['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', ':', ';', '<', '=', '>', '?', '@', '[', '\\', ']', '^', '_', '`', '{', '|', '}', '~', ' '], max_str_len=None, vocab=<torchtext.vocab.Vocab object at 0x7f4c01edeb50>, n_channels=8, embed_dropout_p=0.2, use_attention=True, use_mask=True),
 'manufacturer': NumericalizeInfo(field_type=<FieldType.MULTITOKEN: 'multitoken'>, tokenizer='entity_embed.data_utils.numericalizer.default_tokenizer', alphabet=['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y'

In [11]:
datamodule = benchmark.build_pairwise_datamodule(
    row_numericalizer=row_numericalizer,
    batch_size=10,
    row_batch_size=16,
    random_seed=random_seed
)

## Training

In [12]:
from entity_embed import LinkageEmbed
from entity_embed.miners import SameBlockMiner

ann_k = 100
model = LinkageEmbed(
    datamodule,
    ann_k=ann_k,
    embedding_size=300,
    miner_cls=SameBlockMiner,
    miner_kwargs={
        'true_pair_set': benchmark.train_true_pair_set,
        'false_pair_set': benchmark.train_false_pair_set,
    }
)

In [13]:
import pytorch_lightning as pl
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning.loggers import TensorBoardLogger

max_epochs = 100
early_stop_callback = EarlyStopping(
   monitor='valid_f1_at_0.7',
   min_delta=0.00,
   patience=20,
   verbose=True,
   mode='max'
)
tb_log_dir = '../tb_logs'
tb_name = 'f1-amzn-googl'
trainer = pl.Trainer(
    gpus=1,
    max_epochs=max_epochs,
    check_val_every_n_epoch=1,
    callbacks=[early_stop_callback],
    logger=TensorBoardLogger(tb_log_dir, name=tb_name)
)

11:39:47 INFO:GPU available: True, used: True
11:39:47 INFO:TPU available: None, using: 0 TPU cores
11:39:47 INFO:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


In [14]:
trainer.fit(model, datamodule)

11:39:49 INFO:
  | Name        | Type           | Params
-----------------------------------------------
0 | blocker_net | BlockerNet     | 4.8 M 
1 | losser      | SupConLoss     | 0     
2 | miner       | SameBlockMiner | 0     
-----------------------------------------------
3.1 M     Trainable params
1.7 M     Non-trainable params
4.8 M     Total params


11:39:50 WARNING:true_pair_set has 82 less elements than transitive_true_pair_set. This means there are transitive true pairs not included in true_pair_set.


1

In [15]:
model.blocker_net.get_signature_weights()

{'title': 0.46854373812675476,
 'manufacturer': 0.17905206978321075,
 'price': 0.3524041771888733}

In [16]:
from entity_embed import validate_best

validate_best(trainer)

{'valid_f1_at_0.3': 0.0338958180484226,
 'valid_f1_at_0.5': 0.3132704858593184,
 'valid_f1_at_0.7': 0.7392290249433106,
 'valid_f1_at_0.9': 0.17829457364341086,
 'valid_pair_entity_ratio_at_0.3': 29.312910284463896,
 'valid_pair_entity_ratio_at_0.5': 2.5054704595185995,
 'valid_pair_entity_ratio_at_0.7': 0.45295404814004375,
 'valid_pair_entity_ratio_at_0.9': 0.0525164113785558,
 'valid_precision_at_0.3': 0.017243953418931025,
 'valid_precision_at_0.5': 0.188646288209607,
 'valid_precision_at_0.7': 0.7874396135265701,
 'valid_precision_at_0.9': 0.9583333333333334,
 'valid_recall_at_0.3': 0.9871794871794872,
 'valid_recall_at_0.5': 0.9230769230769231,
 'valid_recall_at_0.7': 0.6965811965811965,
 'valid_recall_at_0.9': 0.09829059829059829}

## Testing

In [17]:
trainer.test(ckpt_path='best', verbose=False)

[{'test_f1_at_0.3': 0.03147451248717072,
  'test_f1_at_0.5': 0.35815899581589955,
  'test_f1_at_0.7': 0.7441860465116279,
  'test_f1_at_0.9': 0.178988326848249,
  'test_pair_entity_ratio_at_0.3': 31.19522776572668,
  'test_pair_entity_ratio_at_0.5': 2.0845986984815617,
  'test_pair_entity_ratio_at_0.7': 0.42516268980477223,
  'test_pair_entity_ratio_at_0.9': 0.049891540130151846,
  'test_precision_at_0.3': 0.015993324525415478,
  'test_precision_at_0.5': 0.222684703433923,
  'test_precision_at_0.7': 0.8163265306122449,
  'test_precision_at_0.9': 1.0,
  'test_recall_at_0.3': 0.9829059829059829,
  'test_recall_at_0.5': 0.9145299145299145,
  'test_recall_at_0.7': 0.6837606837606838,
  'test_recall_at_0.9': 0.09829059829059829}]

## Testing manually 

In [18]:
from entity_embed.data_utils.utils import id_pairs_to_cluster_mapping_and_dict, cluster_dict_to_filtered_row_dict

__, test_cluster_dict = id_pairs_to_cluster_mapping_and_dict(benchmark.test_true_pair_set)
test_row_dict = cluster_dict_to_filtered_row_dict(benchmark.row_dict, test_cluster_dict)
test_left_vector_dict, test_right_vector_dict = model.predict(
    row_dict=test_row_dict,
    left_id_set={id_ for id_, row in test_row_dict.items() if row['__source'] == 'left'},
    right_id_set={id_ for id_, row in test_row_dict.items() if row['__source'] == 'right'},
    batch_size=16
)

In [19]:
embedding_size = model.blocker_net.embedding_size

In [20]:
assert (len(test_left_vector_dict) + len(test_right_vector_dict)) == len(test_row_dict)

In [21]:
%%time

from entity_embed import ANNLinkageIndex

ann_index = ANNLinkageIndex(embedding_size=embedding_size)
ann_index.insert_vector_dict(left_vector_dict=test_left_vector_dict, right_vector_dict=test_right_vector_dict)
ann_index.build()

CPU times: user 275 ms, sys: 434 µs, total: 276 ms
Wall time: 43 ms


In [42]:
%%time

sim_threshold = 0.7
found_pair_set = ann_index.search_pairs(
    k=ann_k,
    sim_threshold=sim_threshold,
    left_vector_dict=test_left_vector_dict,
    right_vector_dict=test_right_vector_dict,
)

CPU times: user 342 ms, sys: 0 ns, total: 342 ms
Wall time: 37.7 ms


In [43]:
from entity_embed.evaluation import pair_entity_ratio

pair_entity_ratio(len(found_pair_set), len(test_row_dict))

0.42516268980477223

In [44]:
from entity_embed.evaluation import precision_and_recall, f1_score

precision, recall = precision_and_recall(found_pair_set, benchmark.test_true_pair_set)
precision, recall

(0.8163265306122449, 0.6837606837606838)

In [45]:
f1_score(precision, recall)

0.7441860465116279

In [46]:
false_positives = list(found_pair_set - benchmark.test_true_pair_set)
len(false_positives)

36

In [47]:
false_negatives = list(benchmark.test_true_pair_set - found_pair_set)
len(false_negatives)

74

In [48]:
cos_similarity = lambda a, b: np.dot(a, b)

In [49]:
for (id_left, id_right) in false_negatives[:10]:
    display(
        (
            cos_similarity(test_left_vector_dict[id_left], test_right_vector_dict[id_right]),
            test_row_dict[id_left], test_row_dict[id_right]
        )
    )

(0.52203286,
 {'id': 17,
  'title': 'upg sgms 1000 incremental node',
  'manufacturer': 'sonic - systems - inc .',
  'price': '',
  '__source': 'left'},
 {'id': 3347,
  'title': 'sonicwall gms 1000n - incremental lic upg',
  'manufacturer': 'sonic - systems - inc .',
  'price': '62920 . 89',
  '__source': 'right'})

(0.30961543,
 {'id': 207,
  'title': 'crystal reports 11 professional full product french',
  'manufacturer': 'business objects',
  'price': '909 . 66',
  '__source': 'left'},
 {'id': 2888,
  'title': 'crystal reports xi professional edition complete package',
  'manufacturer': '',
  'price': '537 . 57',
  '__source': 'right'})

(0.5952761,
 {'id': 431,
  'title': 'microsoft windows server 2003 client additional license for devices 5 pack',
  'manufacturer': 'microsoft',
  'price': '209 . 0',
  '__source': 'left'},
 {'id': 4472,
  'title': 'microsoft windows server 2003 client additional license for devices 5 pack ( 823930 )',
  'manufacturer': '',
  'price': '158 . 39',
  '__source': 'right'})

(0.43832397,
 {'id': 1206,
  'title': 'contentbarrier x4 10 . 4 single user ( mac )',
  'manufacturer': 'intego',
  'price': '49 . 99',
  '__source': 'left'},
 {'id': 1900,
  'title': 'intego contentbarrier x4 10 . 4',
  'manufacturer': '',
  'price': '54 . 99',
  '__source': 'right'})

(0.6459907,
 {'id': 143,
  'title': 'adobe acrobat distiller svr v6 - cd linux u / u 42050142 )',
  'manufacturer': 'adobe systems',
  'price': '',
  '__source': 'left'},
 {'id': 3193,
  'title': 'adobe 42050142 acrobat distiller 6 lnx ret cd u / l lnx',
  'manufacturer': '',
  'price': '14967 . 47',
  '__source': 'right'})

(0.42097065,
 {'id': 965,
  'title': 'pinnacle mobile media organizer',
  'manufacturer': 'pinnacle',
  'price': '49 . 99',
  '__source': 'left'},
 {'id': 3677,
  'title': 'pinnacle mobile media organizer software for windows ipod software',
  'manufacturer': '',
  'price': '39 . 95',
  '__source': 'right'})

(0.5971346,
 {'id': 345,
  'title': 'portfolio media kit be syst recovery 7 . 0 win small business ed',
  'manufacturer': 'symantec',
  'price': '50 . 0',
  '__source': 'left'},
 {'id': 2883,
  'title': 'symantec 11859201 be sys recovery 7 . 0 win sbs ed media cd m / l',
  'manufacturer': '',
  'price': '31 . 98',
  '__source': 'right'})

(0.5862651,
 {'id': 764,
  'title': 'agatha christie : and then there were none',
  'manufacturer': 'dreamcatcher interactive',
  'price': '29 . 99',
  '__source': 'left'},
 {'id': 1750,
  'title': 'agatha christie and then there were none e ( pc ) tha christi',
  'manufacturer': '',
  'price': '7 . 99',
  '__source': 'right'})

(0.64713424,
 {'id': 1099,
  'title': 'webroot spysweeper antispyware 3 user',
  'manufacturer': 'webroot software',
  'price': '39 . 99',
  '__source': 'left'},
 {'id': 2130,
  'title': 'webroot software 65210 spy sweeper 3 pc',
  'manufacturer': 'webroot software',
  'price': '31 . 99',
  '__source': 'right'})

(0.576061,
 {'id': 404,
  'title': 'cosmi rom07524 print perfect business cards dvd',
  'manufacturer': 'cosmi',
  'price': '',
  '__source': 'left'},
 {'id': 4253,
  'title': 'print perfect business cards dvd ( pc ) cosmi',
  'manufacturer': '',
  'price': '29 . 99',
  '__source': 'right'})